In [1]:
%load_ext autoreload

In [2]:
import os, sys

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras as keras

In [4]:
BASE_PATH = os.path.join(os.getcwd(), "..", "..", "..")
MODULES_PATH = os.path.join(BASE_PATH, "modules")
DATASET_PATH = os.path.join(BASE_PATH, "datasets")

In [5]:
sys.path.append(MODULES_PATH)

In [6]:
from active_learning import Config, AcquisitionFunction, Pool
from wrapper import McDropout, MomentPropagation
from models import setup_growth, fchollet_cnn
from data import BenchmarkData, DataSetType

In [7]:
mnist_path = os.path.join(DATASET_PATH, "mnist")
dataset = BenchmarkData(DataSetType.MNIST, mnist_path)

In [8]:
setup_growth()

1 Physical GPU's,  1 Logical GPU's


In [9]:
base_model = fchollet_cnn(output=10)

In [10]:
seed = 90231
np.random.seed(seed)
tf.random.set_seed(seed)

### Split data

In [11]:
x_train, x_test, y_train, y_test = train_test_split(dataset.inputs, dataset.targets, test_size=10000)

In [12]:
len(x_train)

60000

In [13]:
pool = Pool(x_train, y_train)

### Define Models

In [14]:
%autoreload 2
fit_params = {"epochs": 2, "batch_size": 10}
compile_params = {"optimizer": "adam", "loss": "sparse_categorical_crossentropy", "metrics": [keras.metrics.SparseCategoricalAccuracy()]}

# Define MC Dropout model
mc_model = McDropout(base_model, config=Config(
    fit=fit_params,
    query={"sample_size": 25},
    evaluate={"sample_size": 25}
))
mc_model.compile(**compile_params)

# Define Moment Propagation model
mp_model = MomentPropagation(base_model, config=Config(
    fit=fit_params
))
mp_model.compile(**compile_params)

In [23]:
mc_model.fit(x_train, y_train)

Epoch 1/2
6000/6000 [==============================] - 17s 3ms/step - loss: 0.1703 - sparse_categorical_accuracy: 0.9491
Epoch 2/2
6000/6000 [==============================] - 16s 3ms/step - loss: 0.0786 - sparse_categorical_accuracy: 0.9772


In [31]:
mp_model.fit(x_train, y_train)

Epoch 1/2
6000/6000 [==============================] - 16s 3ms/step - loss: 0.0607 - sparse_categorical_accuracy: 0.9820
Epoch 2/2
6000/6000 [==============================] - 17s 3ms/step - loss: 0.0499 - sparse_categorical_accuracy: 0.9852


In [15]:
mc_model.evaluate(x_test[:10], y_test[:10])

{'loss': 2.273679, 'accuracy': 0.1}

In [16]:
mp_model.evaluate(x_test[:10], y_test[:10])

{'loss': 2.2762954, 'accuracy': 0.1}

### Try BALD acquisition

In [18]:
mc_bald = mc_model.get_query_fn("bald")
mc_bald(x_train[:100], sample_size=100)

array([0.00209641, 0.00168586, 0.00196457, 0.00379324, 0.00214124,
       0.00196004, 0.00176787, 0.00267005, 0.00246286, 0.00182962,
       0.00274563, 0.00189328, 0.00464845, 0.00244999, 0.00179839,
       0.00290751, 0.00369906, 0.00293708, 0.0013926 , 0.00361896,
       0.00314736, 0.00337267, 0.00278759, 0.0035913 , 0.00298953,
       0.00273037, 0.00248981, 0.00291824, 0.0052259 , 0.00327992,
       0.00248885, 0.00138068, 0.00093818, 0.00144386, 0.00265503,
       0.00245547, 0.0046463 , 0.00435209, 0.00219822, 0.00285196,
       0.00162506, 0.0037744 , 0.00329876, 0.00086665, 0.00212765,
       0.00291276, 0.0045495 , 0.00308037, 0.0036099 , 0.00379062,
       0.00362349, 0.00112319, 0.00405765, 0.00297809, 0.00191569,
       0.0033021 , 0.00279164, 0.003793  , 0.00370455, 0.00157428,
       0.00347328, 0.00240541, 0.00259733, 0.00296855, 0.00294924,
       0.00126314, 0.00164056, 0.00266194, 0.002321  , 0.00190973,
       0.00345969, 0.00268221, 0.00223446, 0.00276184, 0.00376

In [20]:
mp_bald = mp_model.get_query_fn("bald")
mp_bald(x_train[:100], num_samples=100)

array([0.00237621, 0.00165268, 0.00192264, 0.00360994, 0.00212713,
       0.00183214, 0.00171279, 0.00280752, 0.00297937, 0.00197724,
       0.00280976, 0.00207319, 0.00369291, 0.00289948, 0.00192857,
       0.0034801 , 0.00351019, 0.0027469 , 0.00145434, 0.00392829,
       0.00372406, 0.00282721, 0.00282511, 0.00353016, 0.0030396 ,
       0.00291649, 0.00237847, 0.0027941 , 0.00540994, 0.00367605,
       0.00227967, 0.00150782, 0.00086967, 0.00130275, 0.00267019,
       0.00226354, 0.00446372, 0.00457414, 0.00222136, 0.00295883,
       0.00170095, 0.00425909, 0.00353157, 0.00099609, 0.00197148,
       0.00281927, 0.00454613, 0.00320257, 0.00381227, 0.00406784,
       0.00357531, 0.00122683, 0.00386755, 0.00279467, 0.00209988,
       0.0038876 , 0.0031816 , 0.00408793, 0.00377251, 0.00150511,
       0.00317779, 0.00249727, 0.00240666, 0.00289356, 0.00277155,
       0.00145349, 0.00177689, 0.0026517 , 0.00258097, 0.00205553,
       0.00318284, 0.00227043, 0.00219924, 0.00284235, 0.00377

## Manual Calculation

In [66]:
import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Input, Flatten

alt_model = Sequential([
        Conv2D(32, 3, activation=tf.nn.relu, padding="same", input_shape=(28, 28, 1)),
        Conv2D(64, 3, activation=tf.nn.relu, padding="same"),
        MaxPooling2D(),
        Dropout(.25),
        Flatten(),
        Dense(128, activation=tf.nn.relu),
        Dropout(.5),
        Dense(10)        
    ])

In [71]:
%autoreload 2
from mp.MomentPropagation import MP

test_mp = MP()
test_mp_model = test_mp.create_MP_Model(model=alt_model, use_mp=True)

In [72]:
E, V = test_mp_model(x_train[:10])

In [73]:
from scipy.stats import norm

In [98]:
norm_dist = norm(E, V)
sampled_data = norm_dist.rvs(size=(100, 10, 10))

In [99]:
sfmx_probs = tf.keras.activations.softmax(tf.convert_to_tensor(sampled_data))

In [100]:
entropy = np.sum(sfmx_probs*np.log(sfmx_probs), -1)

In [101]:
disagreement = np.mean(entropy, axis=0)

In [102]:
disagreement

array([-2.30162172, -2.30129182, -2.30161925, -2.30201724, -2.30190348,
       -2.30160955, -2.30211419, -2.30157319, -2.30167709, -2.3019599 ])

In [108]:
E_S, V_S = MP.Gaussian_Softmax(E, V)

In [104]:
entropy = np.sum(E_S*np.log(E_S), axis=1)

In [105]:
entropy

array([-2.3016222, -2.301296 , -2.30163  , -2.302037 , -2.3019164,
       -2.301627 , -2.3021183, -2.3015842, -2.3016884, -2.301966 ],
      dtype=float32)

In [106]:
(-entropy + disagreement)

array([4.34272879e-07, 4.18040503e-06, 1.07724302e-05, 1.97578683e-05,
       1.28842519e-05, 1.73669160e-05, 4.11638489e-06, 1.10527488e-05,
       1.13395817e-05, 6.05630197e-06])

### Try with softmax distribution

In [79]:
E_S, V_S = MP.Gaussian_Softmax(E, V)

In [80]:
sampled_data = norm(E_S, V_S).rvs(size=(100, 10, 10))